### EDA Automation Notebook
This notebook is a jupyter notebook template for all my EDA projects. This will contain various checks that I commonly use in my projects. Since this is an automation or a template, this does not contain any test data. It will contain the following checks as seen below:
- Step 1: Loading the data
- Step 2: Checking the basic data information
- Step 3: Data quality assessment
- Step 4: Summary statistics
- Step 5: Correlations and outliers
- Step 6: Answering questions

There will be another python file and this prints out the pdf report instead of the notebook as this is used for data exploration only and not for answering questions. I used ChatGPT to help with the documentation of the code here as well as to identify the better and more efficient code to implement.

In [1]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
from IPython.display import display
import platform
import subprocess

##### Step 1: Loading the data

In [2]:
def load_data(file_path, sheet_name = None, encoding = "utf-8"):
    """
    Load a dataset either from a CSV or Excel file from any path provide by the user.

    Parameters:
        file_path (str): Path to the file, can be absolute or relative.
        sheet_name (str or int, optional): Sheet name is used as index for files.
    
    Output:
        dataframe: Data from file is loaded as a dataframe. Loads 50 rows(head) of data for better checking.
    """
    if len(file_path) == 0:
        raise FileNotFoundError(f"File path cannot be blank.")
    #Error handling for files if file cannot be located
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found for {file_path}. Ensure file is inside the folder or input the whole path instead.")

    ext_type = os.path.splitext(file_path)[-1].lower()

    #File type handling for CSV and Excel files as well
    try:    
        if ext_type == ".csv":
            #Unicode decode error handling
            try:
                df = pd.read_csv(file_path, encoding = encoding)
            except UnicodeDecodeError:
                print("UTF-8 failed, trying ISO-8859-1...")
                df = pd.read_csv(file_path, encoding = "ISO-8859-1")
        
        elif ext_type in [".xls", "xlsx"]:
            df = pd.read_excel(file_path, sheet_name = sheet_name if sheet_name else 0)
        
        #Error raising for unsupported file types
        else:
            raise ValueError("Unsupported file type. This notebook can only load CSV or Excel files.")

    except Exception as exc:
        raise RuntimeError(f"Error reading file: {exc}")

    print(f"Data loaded successfully from {file_path}")
    display(df.head(50))
    return df

In [ ]:
#Interactive data loading for targeted file path loading. This allows to choose any file at any location.
if __name__ == "__main__":
    file_path = input("Enter file path (ex. data.csv or C:/folder/data.csv): ").strip()
    sheet_name = input("Optional: Specify sheet name or press ENTER to load the file directly.")
    sheet_name = sheet_name if sheet_name else None

    df = load_data(file_path, sheet_name)

##### Step 2: Checking the basic data information

In [33]:
def basic_data_information(df, preview_rows = 5):
      """
      This function prints the basic data information of the dataframe including:
      - Shape
      - Data types
      - Missing values
      - Descriptive statistics
      - Unique counts
      - Preview of rows

      Parameters: 
      df (pd.DataFrame): Data to be summarized.
      preview_rows(int): How many rows to preview, can set to 0 to skip.
      """
      #Set higher display limits to prevent results from getting cutoff when printing
      pd.set_option('display.max_columns', 100)   # set max number of columns to show
      pd.set_option('display.max_rows', 100)      # set number of max rows to show
      pd.set_option('display.max_colwidth', None) # set to None to prevent cell content from truncating
      numeric_cols = df.select_dtypes(include = ['number']).shape[1]
      non_numeric_cols = df.select_dtypes(exclude = ['number']).shape[1]

      print("=" * 110 + "\n" +"Basic Data Information: " + "\n" +
            f"There are {df.shape[0]:,} rows and {df.shape[1]:,} columns in the data." + "\n" +
            "Here are the data types per column:")
      display(df.dtypes.to_frame('Column data types'))
      print(f"There are {numeric_cols} numeric columns and {non_numeric_cols} non-numeric columns in the dataset.")
            

      print("\n" + "-" * 110 + "\n" * 2 + "Missing Values: ")
      mv = df.isnull().sum()
      mv = mv[mv > 0].sort_values(ascending = False)
      if not mv.empty:
            display(mv.to_frame('count_of_missing_values'))
      else:
            print("No missing values in the dataset.")

      print("\n" + "-" * 110 + "\n" * 2 + "Descriptive Statistics: ")
      display(df.describe(include = 'all').T)

      print("\n" + "-" * 110 + "\n" * 2 + "Unique items per column: ")
      c_uniques = df.nunique().sort_values(ascending = False)
      display(c_uniques.to_frame('unique_values'))

      if preview_rows:
            print(f"Data preview: first {preview_rows} rows")
            display(df.head(preview_rows))

      print("\n" + "=" * 110 + "\n" + f"End of summary for {file_path}." 
            + "\n" + "=" * 110)

In [ ]:
basic_data_information(df)

##### Step 3: Data quality assessment

Data quality assessment offers an deep dive on the quality of the data. While some methods are similar with that on step 2, step 3 will have a deeper quality check to ensure that the data is 'ready' for further analysis.

Adding in a functionality that prints the output to a pdf instead of the console to review the outputs for large datasets.

In [ ]:
def data_quality_deep_dive(df, preview_rows=5, size_threshold=300, force_mode=None, file_prefix="quality_deep_dive_summary"):
    """
    Perform a deeper dive in terms of quality assessment on the dataset specifically with:
    - Duplicates
    - Mixed data types
    - Capitalization inconsistencies
    - Unexpected values
    - Optionally export to a pdf file if the output is too large

    Parameters:
    df (pd.DataFrame): Dataframe loaded from steps above.
    preview_rows (int): Max number of rows to preview.
    size_threshold (int): Max row or columns before triggering file output.
    force_mode (str): Allows the function to determine whether to "print", "file", or None (auto-detect based on size).
    file_prefix (str): Name prefix for output file if file mode is used.
    """

    #Identify if output will be printed in the results or exported to a file
    mode = force_mode
    if mode is None: 
        if df.shape[0] > size_threshold or df.shape[1] > size_threshold:
            mode ="file"
        else:
            mode = "print"
    output_lines = []

    def add_output(line):
        if mode == "print":
            print(line)
        else:
            output_lines.append(str(line))
    
    def add_df_output(sub_df, title):
        if mode == "print":
            print("\n" + "=" * 100 + f"\n{title}: \n")
        else:
            output_lines.append(f"\n\n{title}: \n")
            output_lines.append(sub_df.to_string())

    add_output("=" * 110)
    add_output(f"DATA QUALITY ASSESSMENT FOR {file_path} with {df.shape[0]:,} rows x {df.shape[1]:,} columns")
    add_output("=" * 110)

    #Placeholder creations
    title_items = ['Duplicate Rows', 
                'Mixed Data Types', 
                'Capitalization Inconsistencies',
                'Unexpected/Unknown Values',
                'Numeric Outliers or Skew Checks',
                'Data Preview'
                ]

    for index, title in enumerate(title_items, 1):
        add_output("\n" + "-" * 100 + f"[{index}] {title}:" + "\n" + "-" * 100)
        
        if index == 1:  # Duplicate Rows
            duplicate_rows = df[df.duplicated()]
            if not duplicate_rows.empty:
                add_output(f"There are {len(duplicate_rows):,} duplicate rows in the dataset.\n")

                #Print duplicate row indices - limited for large datasets
                duplicate_indices = duplicate_rows.index.tolist()
                add_output("Duplicate row indices (showing up to 100):")
                add_output(str(duplicate_indices[:100]))  # limit to first 100 only

                if len(duplicate_rows) <= size_threshold:
                    add_output(duplicate_rows.to_string(index=False))
                else:
                    add_output("Too many duplicates to display; showing top 5:")
                    add_output(duplicate_rows.head(5).to_string(index=False))
            else:
                add_output("No exact duplicate rows found.")

        elif index == 2: #Mixed data types
            mixed_data_types_issues = {}

            for col in df.columns:
                mixed_types = df[col].dropna().map(type).value_counts()

                if len(mixed_types) > 1:
                    mixed_data_types_issues[col] = mixed_types

            if mixed_data_types_issues:
                add_output(f"Found {len(mixed_data_types_issues)} columns with mixed data types:\n")
                for col, types in mixed_data_types_issues.items():
                    add_output(f"- Column '{col}' has multiple data types:")
                    for t, count in types.items():
                        add_output(f"    • {t.__name__}: {count:,} values")
            else:
                add_output("No mixed data types found in all columns.")
            
        elif index == 3: #Capitalization inconsistencies 
            return None
        elif index == 4: #Unexpected/unknown values 
            return None
        elif index == 5: #Numeric outliers or skew checks
            return None
        elif index == 6: #Data preview
            return None
        
        # Final output
    if mode == "file":
        output_file = f"{file_prefix}.txt"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write("\n".join(output_lines))
        print(f"\nData quality report written to {output_file}")


##### Step 4: Summary statistics

##### Step 5: Correlations and outliers

##### Step 6: Answering questions